In [1]:
import pandas as pd

K = 25

In [2]:
def parser(file_name, sheet_name):

    df = pd.read_excel(file_name, sheet_name=sheet_name, header=None)
    df.loc[len(df)] = [None, None]
    matches = []
    player_names = []
    scores = []

    for i, row in df.iterrows():
        player = row[0]  # Column A (Player)
        score = row[1]   # Column B (Score)

        if pd.isna(player) and pd.isna(score):
            if len(player_names) in [2, 3, 4, 5]:
                matches.append({'player_names': player_names, 'scores': scores})
            
            player_names = []
            scores = []
            continue

        player_names.append(player)
        scores.append(score)
    print(f"{sheet_name} total number of matches is: {len(matches)}")
    return matches

In [3]:
# returns expected score for player A with rating Ra
# expeced score is win probability + 0.5 * draw probability
def calculate_expected_score(Ra, Rb):
    Ea = 1 / (1 + pow(10, (Rb-Ra)/400))
    return Ea

In [4]:
# (player_id : name)
players = pd.read_excel("Data.xlsx", sheet_name="Players", header=None)
players.columns = players.iloc[0]
players = players.drop(0).reset_index(drop=True)

# Convert to dictionary (ID as key, Player as value)
id_to_player_dict = pd.Series(players['Player'].values, index=players['ID']).to_dict()

player_to_id_dict = {v: k for k, v in id_to_player_dict.items()}
    
# (player_id : rating)
ratings_dict = {i: 1500 for i in range(1, len(id_to_player_dict)+1)}

print(player_to_id_dict['Сергей Лобачёв'])
print(player_to_id_dict['Мария Корчемкина'])

23
37


In [ ]:
# we'll need a dictionary with current ratings, something like a database for 100+ players
# we also need to have a function that calculates updated ratings based on a match

# gets a list of players (in fact player ids and corresponding points in a match and then
# updates elo for all of them

def update_elo(players, points):
    expected_scores = [0] * len(players)
    actual_scores = [0] * len(players)
    for idx_A, player_A in enumerate(players):
        for idx_B, player_B in enumerate(players):
            if idx_A != idx_B:
                expected_scores[idx_A] += calculate_expected_score(ratings_dict[player_A], ratings_dict[player_B])
                actual_scores[idx_A] += 1 * int(points[idx_A] > points[idx_B]) + 0.5 * int(points[idx_A] == points[idx_B])
    
    for idx_A, player in enumerate(players):
        ratings_dict[player] += K * (actual_scores[idx_A] - expected_scores[idx_A])
    return expected_scores, actual_scores

tournament_sequence=[
    "Vuelta_1",
    "USA",
    "Bez_Povoda",
    "Poligon",
    "Vuelta_2",
    "Nevermore_1",
    "Nevermore_2"
]

for tournament in tournament_sequence:
    matches = parser("Data.xlsx", tournament)
    for match in matches:
        players = [player_to_id_dict[player] for player in match['player_names']]
        scores = match['scores']
        update_elo(players, scores)
    
sorted_rating = [[id_to_player_dict[id], ratings_dict[id]] for id in id_to_player_dict]
sorted_rating = sorted(sorted_rating, key = lambda x: -x[1])
for player, elo in sorted_rating:
    print(f"{player} : {elo:.0f}")

Vuelta_1 total number of matches is: 48
USA total number of matches is: 55
Bez_Povoda total number of matches is: 42
Poligon total number of matches is: 15


C:\Users\Sergey Lobachev\AppData\Local\Temp\ipykernel_16668\1240587464.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df.loc[len(df)] = [None, None]
C:\Users\Sergey Lobachev\AppData\Local\Temp\ipykernel_16668\1240587464.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df.loc[len(df)] = [None, None]
C:\Users\Sergey Lobachev\AppData\Local\Temp\ipykernel_16668\1240587464.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future vers

Vuelta_2 total number of matches is: 38
Nevermore_1 total number of matches is: 14
Nevermore_2 total number of matches is: 14
Сергей Лобачёв : 1796
Вадим Барановский : 1787
Андрей Романчев : 1764
Юрий Корчемкин : 1754
Шакир Мамедзаде : 1678
Евгений Затуловский : 1653
Николай Арчак : 1634
Владимир Грамагин : 1632
Станислав Алимпов : 1626
Григорий Смыслов : 1617
Илья Миронов : 1613
Алексей Чернов : 1605
Иван Суманеев  : 1598
Евгений Новиков : 1590
Петр Мартынов : 1579
Ирина Соколова : 1577
Леонид Койфман : 1577
Жора Яковлев : 1568
Григорий Львович : 1560
Арнур Нигметов : 1554
Константин Бриф : 1538
Юлия Жоголева : 1537
Роман Козелов : 1527
Слава Бельков : 1524
Сергей Меньшиков : 1522
Макс Галкин : 1520
Виктор Исаев : 1512
Марианна Выдревич : 1508
Тимур Мухаматулин : 1507
Руслан Лепшоков : 1505
Ефим Подвойский : 1503
Илья Устиловский : 1499
Борис Вейцман : 1497
Евгения Никушина : 1480
Мария Корчемкина : 1480
Туриан Рафаэлли : 1477
Владимир Пузырев : 1474
Денис Чебыкин : 1469
Олжас Усипбае

C:\Users\Sergey Lobachev\AppData\Local\Temp\ipykernel_16668\1240587464.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df.loc[len(df)] = [None, None]
